In [ ]:
import h5py as h5
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import csv
import h5py
from torchvision.models import resnet18
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings 
warnings.filterwarnings("ignore")

In [2]:
import kagglehub
path = kagglehub.dataset_download("harshsaini10/deeplense-task1")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\hs106\.cache\kagglehub\datasets\harshsaini10\deeplense-task1\versions\1


In [4]:
train_path = r"C:\Users\hs106\.cache\kagglehub\datasets\harshsaini10\deeplense-task1\versions\1\dataset\train"
train_imgs = {}
for cat in os.listdir(train_path):
    sub_path = os.path.join(train_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    train_imgs[cat] = imgs

In [5]:
val_path = r"C:\Users\hs106\.cache\kagglehub\datasets\harshsaini10\deeplense-task1\versions\1\dataset\val"
val_imgs = {}
for cat in os.listdir(val_path):
    sub_path = os.path.join(val_path,cat)
    imgs = []
    for img_path in os.listdir(sub_path):
        imgs.append(os.path.join(sub_path,img_path))
    val_imgs[cat] = imgs

In [6]:
train_rows = [(img_path, label) for label, img_paths in train_imgs.items() for img_path in img_paths]
val_rows = [(img_path, label) for label, img_paths in val_imgs.items() for img_path in img_paths]

In [7]:
df_train = pd.DataFrame(train_rows, columns = ['img_path','label'])
df_val = pd.DataFrame(val_rows, columns = ['img_path','label'])

In [8]:
df_train['label'] = df_train['label'].map({'no':0, 'sphere':1, 'vort':2})
df_val['label'] = df_val['label'].map({'no':0, 'sphere':1, 'vort':2})

In [9]:
class Dataset(nn.Module):
    def __init__(self,data,transform = None, train = True):
        self.data = data
        self.imgs = [np.load(img_path) for img_path in self.data['img_path']]
        self.y = torch.tensor(self.data['label'], dtype = torch.long)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.imgs[idx].squeeze()[:,:,np.newaxis]
        label = self.y[idx]
        img = np.tile(img,(1,1,3))
        if self.transform != None:
            img = self.transform(image = img)["image"]

        return img, label

In [10]:
train_transform = A.Compose([
    # A.RandomRotate90(p=0.5),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    # A.Normalize(mean=(0.485, 0.456, 0.406), 
    #             std=(0.229, 0.224, 0.225)),
    # A.Resize(28,28),
    ToTensorV2()
])

val_transform = A.Compose([
    # A.Resize(28,28),
    ToTensorV2()
])

In [12]:
train_dataset = Dataset(df_train, train_transform)
val_dataset = Dataset(df_val, val_transform)

In [13]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [ ]:
class AuxLaplacianCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(AuxLaplacianCNN, self).__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 37 * 37, 256), 
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        self.potential_head = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 1, kernel_size=3, padding=1)
        )
    
    def forward(self, x):
        features = self.shared(x)
        class_out = self.classifier(features)
        potential = self.potential_head(features)
        return class_out, potential

def laplacian_loss(potential):
    laplace_kernel = torch.tensor([[0, 1, 0],
                                   [1, -4, 1],
                                   [0, 1, 0]], dtype=torch.float32, device=potential.device).unsqueeze(0).unsqueeze(0)
    laplacian_pred = F.conv2d(potential, laplace_kernel, padding=1)
    return F.mse_loss(laplacian_pred, torch.zeros_like(laplacian_pred))

def get_pretrained_model(model_name, num_classes=3, pretrained=True):
    if model_name.lower() == 'auxlaplaciancnn':
        model = AuxLaplacianCNN(num_classes=num_classes)
        return model
    else:
        try:
            model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)
            return model
        except Exception as e:
            raise ValueError(f"Error creating model '{model_name}' with timm: {e}")

def train_aux_laplacian(model_name, train_dataloader, val_dataloader, num_epochs=10, device='cuda', lambda_lap=0.1):
    model = get_pretrained_model(model_name, num_classes=3, pretrained=True)
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    is_aux_model = hasattr(model, 'potential_head')
    
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        model.train()
        total_loss = 0.0
        total_correct = 0
        total_samples = 0
        
        epoch_true = []
        epoch_preds = []
        
        for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images = images.float().to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            if is_aux_model:
                outputs, potential = model(images)
                cls_loss = criterion(outputs, labels)
                aux_loss = laplacian_loss(potential)
                loss = cls_loss + lambda_lap * aux_loss
                preds = F.softmax(outputs, dim=1)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                preds = F.softmax(outputs, dim=1)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * images.size(0)
            total_samples += labels.size(0)
            _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
            total_correct += predicted.eq(labels).sum().item()

            epoch_true.extend(labels.cpu().numpy())
            epoch_preds.extend(preds.cpu().detach().numpy())

            del images, labels, outputs, preds
            if is_aux_model:
                del potential, aux_loss, cls_loss
        
        try:
            auc = roc_auc_score(epoch_true, epoch_preds, multi_class='ovr')
        except Exception as e:
            auc = float('nan')
            print("AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Train Loss {total_loss/total_samples:.4f}, Train Acc {total_correct/total_samples:.4f}, Train AUC {auc:.4f}")
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0
        val_true = []
        val_preds = []
        with torch.no_grad():
            for images, labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images = images.float().to(device)
                labels = labels.to(device)
                if is_aux_model:
                    outputs, potential = model(images)
                    cls_loss = criterion(outputs, labels)
                    aux_loss = laplacian_loss(potential)
                    loss = cls_loss + lambda_lap * aux_loss
                    preds = F.softmax(outputs, dim=1)
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    preds = F.softmax(outputs, dim=1)
                val_loss += loss.item() * images.size(0)
                total_val += labels.size(0)
                _, predicted = (outputs.max(1) if is_aux_model else torch.max(outputs, 1))
                val_correct += predicted.eq(labels).sum().item()
                val_true.extend(labels.cpu().numpy())
                val_preds.extend(preds.cpu().detach().numpy())

                del images, labels, outputs, preds
                if is_aux_model:
                    del potential, aux_loss, cls_loss
                torch.cuda.empty_cache()
                gc.collect()
                
        try:
            val_auc = roc_auc_score(val_true, val_preds, multi_class='ovr')
        except Exception as e:
            val_auc = float('nan')
            print("Validation AUC calculation error:", e)
        
        print(f"Epoch {epoch+1}: Val Loss {val_loss/total_val:.4f}, Val Acc {val_correct/total_val:.4f}, Val AUC {val_auc:.4f}")
    return model

In [36]:
train_aux_laplacian('efficientnet_b0', train_loader, val_loader)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b0.ra_in1k)
INFO:timm.models._hub:[timm/efficientnet_b0.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Epoch 1/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:56<00:00,  8.09it/s]


Epoch 1: Train Loss 1.1295, Train Acc 0.4571, Train AUC 0.6458


Epoch 1/10 [Val]: 100%|██████████████████████████████████████████████████████████████| 235/235 [01:14<00:00,  3.17it/s]


Epoch 1: Val Loss 0.7934, Val Acc 0.6301, Val AUC 0.8133


Epoch 2/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:54<00:00,  8.21it/s]


...

Epoch 9: Val Loss 0.2568, Val Acc 0.9116, Val AUC 0.9831


Epoch 10/10 [Train]: 100%|███████████████████████████████████████████████████████████| 938/938 [01:55<00:00,  8.12it/s]


Epoch 10: Train Loss 0.1629, Train Acc 0.9435, Train AUC 0.9912


Epoch 10/10 [Val]: 100%|█████████████████████████████████████████████████████████████| 235/235 [01:12<00:00,  3.23it/s]

Epoch 10: Val Loss 0.4393, Val Acc 0.8223, Val AUC 0.9744


In [38]:
model = train_aux_laplacian('efficientnet_b1', train_loader, val_loader)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnet_b1.ft_in1k)
INFO:timm.models._hub:[timm/efficientnet_b1.ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Epoch 1/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [02:36<00:00,  6.00it/s]


Epoch 1: Train Loss 1.0777, Train Acc 0.4987, Train AUC 0.6868


Epoch 1/10 [Val]: 100%|██████████████████████████████████████████████████████████████| 235/235 [01:24<00:00,  2.78it/s]


Epoch 1: Val Loss 0.6824, Val Acc 0.6916, Val AUC 0.8761


Epoch 2/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [02:28<00:00,  6.33it/s]


...

Epoch 9: Val Loss 0.2290, Val Acc 0.9201, Val AUC 0.9832


Epoch 10/10 [Train]: 100%|███████████████████████████████████████████████████████████| 938/938 [01:52<00:00,  8.31it/s]


Epoch 10: Train Loss 0.1189, Train Acc 0.9585, Train AUC 0.9952


Epoch 10/10 [Val]: 100%|█████████████████████████████████████████████████████████████| 235/235 [01:15<00:00,  3.12it/s]

Epoch 10: Val Loss 0.2661, Val Acc 0.9100, Val AUC 0.9812


In [39]:
model = train_aux_laplacian('resnet18', train_loader, val_loader)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
INFO:timm.models._hub:[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Epoch 1/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:40<00:00,  9.34it/s]


Epoch 1: Train Loss 0.9957, Train Acc 0.4518, Train AUC 0.6595


Epoch 1/10 [Val]: 100%|██████████████████████████████████████████████████████████████| 235/235 [01:17<00:00,  3.02it/s]


Epoch 1: Val Loss 1.9285, Val Acc 0.4305, Val AUC 0.7423


Epoch 2/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:30<00:00, 10.35it/s]


...

Epoch 9: Val Loss 0.2513, Val Acc 0.9107, Val AUC 0.9816


Epoch 10/10 [Train]: 100%|███████████████████████████████████████████████████████████| 938/938 [01:25<00:00, 11.02it/s]


Epoch 10: Train Loss 0.1403, Train Acc 0.9489, Train AUC 0.9933


Epoch 10/10 [Val]: 100%|█████████████████████████████████████████████████████████████| 235/235 [01:14<00:00,  3.14it/s]

Epoch 10: Val Loss 0.2661, Val Acc 0.9100, Val AUC 0.9811


In [40]:
torch.save({'model_state_dict':model.state_dict()},'resnet18_laplacian_loss.pth')

In [41]:
model = train_aux_laplacian('densenet121', train_loader, val_loader)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/densenet121.ra_in1k)


model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

C:\Users\hs106\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hs106\.cache\huggingface\hub\models--timm--densenet121.ra_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
INFO:timm.models._hub:[timm/densenet121.ra_in1k] Safe alternative available for 'pyto

Epoch 1: Train Loss 1.0992, Train Acc 0.3653, Train AUC 0.5403


Epoch 1/10 [Val]: 100%|██████████████████████████████████████████████████████████████| 235/235 [01:23<00:00,  2.80it/s]


...

Epoch 9: Val Loss 0.2178, Val Acc 0.9208, Val AUC 0.9848


Epoch 10/10 [Train]: 100%|███████████████████████████████████████████████████████████| 938/938 [02:56<00:00,  5.32it/s]


Epoch 10: Train Loss 0.1798, Train Acc 0.9349, Train AUC 0.9893


Epoch 10/10 [Val]: 100%|█████████████████████████████████████████████████████████████| 235/235 [01:20<00:00,  2.94it/s]

Epoch 10: Val Loss 0.2332, Val Acc 0.9212, Val AUC 0.9838


In [42]:
torch.save({'model_state_dict':model.state_dict()},'densenet121_laplacian_loss.pth')

In [44]:
model = train_aux_laplacian('mobilenetv2_100', train_loader, val_loader)

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/mobilenetv2_100.ra_in1k)
INFO:timm.models._hub:[timm/mobilenetv2_100.ra_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (classifier.weight, classifier.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Epoch 1/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:34<00:00,  9.94it/s]


Epoch 1: Train Loss 1.1553, Train Acc 0.4328, Train AUC 0.6147


Epoch 1/10 [Val]: 100%|██████████████████████████████████████████████████████████████| 235/235 [01:19<00:00,  2.95it/s]


Epoch 1: Val Loss 0.9949, Val Acc 0.5009, Val AUC 0.7025


Epoch 2/10 [Train]: 100%|████████████████████████████████████████████████████████████| 938/938 [01:58<00:00,  7.93it/s]


...

Epoch 9: Val Loss 0.3188, Val Acc 0.8821, Val AUC 0.9730


Epoch 10/10 [Train]: 100%|███████████████████████████████████████████████████████████| 938/938 [01:35<00:00,  9.81it/s]


Epoch 10: Train Loss 0.2042, Train Acc 0.9249, Train AUC 0.9869


Epoch 10/10 [Val]: 100%|█████████████████████████████████████████████████████████████| 235/235 [01:18<00:00,  2.99it/s]

Epoch 10: Val Loss 0.2939, Val Acc 0.8960, Val AUC 0.9758


In [45]:
torch.save({'model_state_dict':model.state_dict()},'mobilenetv2_laplacian_loss.pth')